https://towardsdatascience.com/genai-with-python-coding-agents-765406d26584

In [1]:
#from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
#llm = ChatGroq(model="llama3-70b-8192")
#llm_id='groq/llama-3.1-8b-instant'

llm = ChatOllama(model="qwen2.5:3b", temperature=0.)
llm_id = "ollama/qwen2.5:3b"

In [2]:
import pandas as pd

dtf = pd.read_csv("train.csv")
dtf.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
len(dtf[dtf["Survived"]==0]) 
#> 549

549

In [4]:
from langchain_community.llms import Ollama #0.2.12

#llm = Ollama(model="llama3.1")
res = llm.invoke(input=['''
Do you know the Titanic dataset?
If yes, tell me how many people died.
'''
])
print(res)

content="Yes, I'm familiar with the Titanic dataset. The number of people who died in the Titanic disaster is 1,317 out of approximately 2,400 passengers and crew members. However, please note that this information can vary slightly depending on the specific source used for data collection and analysis." response_metadata={'model': 'qwen2.5:3b', 'created_at': '2024-09-25T00:36:32.102001068Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2457625900, 'load_duration': 15146888, 'prompt_eval_count': 46, 'prompt_eval_duration': 279342000, 'eval_count': 64, 'eval_duration': 2025184000} id='run-04d6caaf-b14a-4adf-8144-ea22b4503f9c-0' usage_metadata={'input_tokens': 46, 'output_tokens': 64, 'total_tokens': 110}


In [5]:
import sqlite3

dtf.to_sql(index=False, name="titanic", 
           con=sqlite3.connect("database.db"), 
           if_exists="replace")

891

In [6]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///database.db")

In [7]:
from crewai_tools import tool #0.8.3
import crewai #0.51.1

In [8]:
## DB Connection
from langchain_community.tools.sql_database.tool import ListSQLDatabaseTool, InfoSQLDatabaseTool, QuerySQLDataBaseTool

@tool("tool_tables")
def tool_tables() -> str:
    """Get all the tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

@tool("tool_schema")
def tool_schema(tables: str) -> str:
    """Get table schema. Example Input: table1, table2, table3"""
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

@tool("tool_query")
def tool_query(sql: str) -> str:
    """Execute a SQL query"""
    return QuerySQLDataBaseTool(db=db).invoke(sql)

print("--- Get Tables ---")
print( tool_tables.run() )

print("\n--- Get Schema ---")
print( tool_schema.run( tables=tool_tables.run() ) )

print("\n--- SQL Query ---")
print( tool_query.run(f"SELECT * FROM {tool_tables.run()} LIMIT 3") )

--- Get Tables ---
Using Tool: tool_tables
titanic

--- Get Schema ---
Using Tool: tool_tables
Using Tool: tool_schema

CREATE TABLE titanic (
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/

--- SQL Query ---
Using Tool: tool_tables
Using Tool: tool_query
[(1, 0, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0, 'A/5 21171', 7.25, None, 'S'), (2, 1, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 38.0, 1, 0, 'PC 17599', 71.2833, 'C85', 'C'), (3, 1, 3, 'Hei

In [9]:
## LLM Checking SQL
from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool

@tool("tool_check")
def tool_check(sql: str) -> str:
    """
    Before executing a query, always use this tool to review the SQL query 
    and correct the code if necessary.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query":sql})

tool_check.run(f"SELECT * FROM {tool_tables.run()} LIMIT 3 +db+") #<--SQL with error

Using Tool: tool_tables
Using Tool: tool_check


'SELECT * FROM titanic LIMIT 3 +db+ \n\nThe provided SQL query contains a syntax error due to the presence of `+db+` within the `LIMIT` clause. In SQLite (and most SQL databases), the `LIMIT` clause should not contain any expressions or variables that are not part of a constant value. Therefore, we cannot correct this specific issue without additional context about what `+db+` might represent.\n\nIf `+db+` is meant to be replaced by a number for limiting the rows (for example, if `3 + db+` evaluates to an integer), then it should be removed and replaced with that integer. If not, the query as written cannot be executed due to the syntax error.\n\nGiven this context, if we assume that `+db+` is intended to replace a number such as 5 (for instance):\n\n```sql\nSELECT * FROM titanic LIMIT 5;\n```\n\nThis final query would retrieve all columns from the table named "titanic" and limit the result set to 5 rows. Please replace or validate `+db+` with an appropriate integer value if needed for

In [10]:
prompt = '''Extract data with SQL query to answer {user_input}'''

## Agent
agent_sql = crewai.Agent(
    role="Database Engineer",
    goal=prompt,
    backstory='''
        You are an experienced database engineer that creates and optimize efficient SQL queries.
        Use the `tool_tables` to find tables.
        Use the `tool_schema` to get the metadata for the tables.
        Use the `tool_check` to review your queries before executing.
        Use the `tool_query` to execute SQL queries.
     ''',
    tools=[tool_tables, tool_schema, tool_query, tool_check], 
    max_iter=10,
    llm=llm_id,
    allow_delegation=False, verbose=True)

## Task
task_sql = crewai.Task(
    description=prompt,
    agent=agent_sql,
    expected_output='''Output of the query''')

In [11]:
crew = crewai.Crew(agents=[agent_sql], tasks=[task_sql], verbose=False)
res = crew.kickoff(inputs={"user_input": "how many people died?"})

# Agent: Database Engineer
## Task: Extract data with SQL query to answer how many people died?


In [ ]:
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool

tool_pycode = Tool(name="tool_pycode",
    description='''
    A Python shell. Use this to execute python commands. 
    Input should be a valid python command. 
    If you want to see the output of a value, 
    you should print it out with `print(...)`.
    ''',
    func=PythonREPL().run)

print( tool_pycode.run("import numpy as np; print(np.sum([1,2]))") ) 
#> 3

In [ ]:
@tool("tool_eval")
def tool_eval(code: str) -> str:
    """
    Before executing Python code, always use this tool to evaluate code and correct the code if necessary.
    Example: `import numpy as np print(np.sum([1,2]))` would give an error, 
    so you must change it to `import numpy as np; print(np.sum([1,2]))`
    """
    res = llm.invoke(input=[f'''review the following python code and correct it if you find errors.
    You must return only the correct code: {code}'''])#.split("\n")[0]
    return res

print( tool_eval.run("importt numpy as np") ) #<--Python with error
#> `import numpy as np`

In [ ]:
prompt = '''You receive data from the database engineer and analyze it with Python to answer {user_input}'''

## Agent
agent_py = crewai.Agent(
    role="Data Analyst",
    goal=prompt,
    backstory='''
        You are an experienced data anlyst that analyzes datasets using Python.
        You have attention to detail and always produce very clear and detailed results.
        First generate Python code you need to analyze the data.
        Then use the `tool_eval` to check your code.
        Finally use `tool_pycode` to execute the code and return the output.
     ''',
    tools=[tool_eval, tool_pycode],
    max_iter=10,
    llm=llm_id,
    allow_delegation=False, verbose=True)

## Task
task_py = crewai.Task(
    description=prompt,
    agent=agent_py,
    context=[task_sql],
    expected_output='''Output of Python code''')

In [ ]:
crew = crewai.Crew(agents=[agent_py], tasks=[task_py], verbose=False)
res = crew.kickoff(inputs={"user_input": 
f"how many people died in this dataset? {dtf.to_string()}"})

In [ ]:
prompt = '''You write executive summary reports based on the work of the data analyst to answer {user_input}'''

## Agent
agent_html = crewai.Agent(
    role="Web Developer",
    goal=prompt,
    backstory='''
        You are an experienced web developer that writes beautiful reports using HTML and CSS.
        You always summarize texts into bullet points containing the most important details.
        At the end add an interactive button with JavaScript so the user can approve the report,
        and if the user clicks the button, show a pop-up text.
     ''',
    max_iter=10,
    llm=llm,
    allow_delegation=False, verbose=True)

## Task
task_html = crewai.Task(
    description=prompt,
    agent=agent_html,
    context=[task_py],
    expected_output='''HTML code''')

In [ ]:
crew = crewai.Crew(agents=[agent_html], tasks=[task_html], verbose=False)
res = crew.kickoff(inputs={"user_input":
'''
Number of people who died: 549
'''})


In [ ]:
crew = crewai.Crew(agents=[agent_sql, agent_py, agent_html], 
                   tasks=[task_sql, task_py, task_html], 
                   process=crewai.Process.sequential,
                   verbose=True)

res = crew.kickoff(inputs={"user_input":"how many people died?"})

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=llm, df=dtf, verbose=True, allow_dangerous_code=True)
agent.invoke('''
        You are an experienced data scientist that does machine learning using Python and sckit-learn.
        Take the dataframe and split into train set and test set. 
        Then train a simple classification to predict the column `Survived`.
        Then use the score to evaluate the model predictions.''')